# Testing rotation of detector to change cake position in pyFAI

This notebook uses the test image found at: http://www.silx.org/pub/pyFAI/testimages/mock.tif
This is clearly an unrealistic scattering image but provides a good way to
show how different detector alignments affect the final data. 

*Note, this image is "mirrored" so that it will display properly when the origin is set to the **bottom left** of the image. Further explanation about the orientation of this image can be found [here]( https://pyfai.readthedocs.io/en/master/usage/tutorial/Introduction/introduction.html)*

We look at a workflow for caking scattering spectra using pyFAI and then reading this data into `xrdfit`.

In [ ]:
import fabio
import xrdfit
from pyFAI.gui import jupyter
import pyFAI.detectors, pyFAI.azimuthalIntegrator
import math
import numpy as np
import matplotlib.pyplot as plt
from xrdfit.spectrum_fitting import FitSpectrum

Load test image from file and note the 

In [ ]:
np.shape(img)

In [ ]:
img = fabio.open(r"example-data/moke.tif").data
jupyter.display(img, label = "Diffraction image - bottom left origin")
print("Note, this image appears mirrored from top to bottom, as the origin is at the bottom left.")

Set up mock detector *- normally you would load a calibration here.*
The parameters are those given with the sample image.

In [ ]:
detector = pyFAI.detectors.Detector(pixel1=1e-4, pixel2=1e-4)
ai = pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=0.1, detector=detector)
# Center the detector on the center of the pattern at 300, 300.
ai.setFit2D(100, 300, 300)

Now rotate the detector. `rot3` is the angle to rotate the detector in the axis of the imaging beam.
See [here](https://pyfai.readthedocs.io/en/latest/usage/tutorial/Geometry/geometry.html#Conclusion) for a
schematic of the detector and rotations.

The minimum cake angle in pyFAI is -180${^\circ}$, and so caking will slice along the horizontal, rather than being centred on the horizontal. We can centre the cake on the horizontal (and vertical) by applying a rotation to the detector. This angle needs to be **negative for caking clockwise** and **positive for caking anti-clockwise**, to rotate the detector in the correct sense.

*Note, with 36 cakes and 100 slices the resolution is poor but xrdfit plotting is fast.
With 360 cakes and 1000 slices it is easier to see image reconstructed from the cakes but
xrdfit plotting is slow. This appears to be a limitation of the radial plotting in matplotlib.*

You can alter the number of cakes, radial slices and detector rotation here to see how it affects the
caked data.

In [ ]:
num_cakes = 72
num_radial_slices = 1000

# Rotate the detector so that the cardinal direction is in the center of the first cake.
cake_angle = 360 / num_cakes
ai.rot3 = - (cake_angle / 2) * (math.pi / 180) # convert rotation to radians

Display the caked image in a flat 2D perspective.

In [ ]:
res2d = ai.integrate2d(img, num_radial_slices, num_cakes, unit="2th_deg")
jupyter.plot2d(res2d, label="moke")

From the image above it appears that pyFAI cakes the data anticlockwise (from left to bottom to right to top, or from -180${^\circ}$ to 0${^\circ}$ to +180${^\circ}$). And this is how the azimuthal angle range is recorded in the `res2d` object - from -180${^\circ}$ to 0${^\circ}$ to +180${^\circ}$).

However, the image above is actually being mirrored due to `jupyter.display` having the origin at the bottom left of the plot. The true image in any usual image viewer (where the origin of the image is in it's usual place at the top left) would look like the one below;

In [ ]:
true_img = fabio.open(r"example-data/moke.tif").data
plt.imshow(true_img)

This is the image we want to replicate in xrdfit, not the mirrored image as it appeared at the start of this notebook using `jupyter.display()`. We can't avoid `jupyter.display()` as it is built into the pyFAI package, so we need to remember we are always viewing a flipped / mirrored image.

And we can now see pyFAI is actually caking clockwise (from left to top to right to bottom).

This is the same clockwise direction used as the default in [DAWN](https://dawnsci.org). However, pyFAI starts caking the data from the left or West direction, whereas DAWN starts caking from the right or East direction. So the angle of the first cake in pyFAI is 270 degrees clockwise of the North direction - which we need to specify when loading the data into `xrdfit`.

By default, `np.savetxt` uses a space as a column delimiter. `xrdfit` defaults to tab spacing but this can be changed using the `delimiter` argument.

We can use pyFAI to reproduce the equivalent output of DAWN as:

In [ ]:
# Transpose to order the intensity data
intensity = res2d.intensity.T

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270, delimiter=" ")
spectrum.plot_polar()

*Note, the color change between images is not important - it is just that `xrdfit` is using a just a different matplotlib colormap to the `jupyter.plot2d` function.*

We can also flip the data to cake anticlockwise if we wanted to, which recovers the original "mirrored" image we showed at the start...

*Note, to do this properly we would need to rotate the data `rot3` in the opposite positive sense. However, for now, to read the anticlockwise data, we will just displace the `first_cake_angle` by one cake width.*

In [ ]:
# Flip the intensity data to order cakes clockwise rather than anticlockwise
intensity = np.flip(res2d.intensity.T, axis=1)

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270+cake_angle, delimiter=" ")
spectrum.plot_polar()

To load this flipped data we could use the new anticlockwise fit option of `xrdfit`.
This is particularly useful for users who have already gone through the process of caking their data using pyFAI, in an anticlockwise sense using the flip function, but would still like to load it as it appears into xrdfit.

In [ ]:
# Flip the intensity data to order cakes clockwise rather than anticlockwise
intensity = np.flip(res2d.intensity.T, axis=1)

# Reshape radial labels to 2D array so they can be attached to the intensity data.
radial = np.reshape(res2d.radial, (-1, 1))

result = np.hstack((radial, intensity))
np.savetxt("example_caked_spectrum.txt", result)
spectrum = xrdfit.spectrum_fitting.FitSpectrum("example_caked_spectrum.txt", first_cake_angle=270-cake_angle, delimiter=" ", cake_order="anticlockwise")
spectrum.plot_polar()